In [1]:
import pandas as pd
import pickle

In [2]:
f = open('../../rawdata/sr5502data.txt', "r")
init_list = f.read().split('\n')
init_list[:5]

['Mitsubishi Mirage hatchback 72 (28-115) 52 18 5 2014-17 171,842',
 'Kia Rio 87 (40-134) 51 38 0 2014-17 204,326',
 'Hyundai Accent 116 (74-158) 85 28 9 2014-17 417,171',
 'Ford Fiesta 141 (94-189) 98 46 13 2014-17 357,492',
 'Volkswagen Golf 0 (0-34) 0 0 0 2015-17 108,084']

In [3]:
#init empty lists and fill them line by line
'''
model_names: name of the car model, and variant if applicable
death_rate: driver death rate per million registered vehicle years
death_rate_ci: 95% confidence interval assuming Poisson distribution of driver death rates
mv: deaths caused by multi-vehicle accidents
sv: deaths caused by single-vehicle accidents
sv_roll: subset of sv, deaths caused by single-vehicle rollover accidents
model_years: model years included in the study
registered_years: total number of vehicle-registered years for the model included in the study 
'''
model_names = []
death_rate = []
death_rate_ci = []
mv = []
sv = []
sv_roll = []
model_years = []
registered_years = []

for line in init_list:
    temp = line.split(' ')

    model_names.append(" ".join(temp[:-7]))
    death_rate.append(temp[-7])
    death_rate_ci.append(temp[-6][1:-1])
    mv.append(temp[-5])
    sv.append(temp[-4])
    sv_roll.append(temp[-3])
    model_years.append(temp[-2])
    registered_years.append(temp[-1])

In [4]:
# Make dataframe with the lists
iihs_raw_death = pd.DataFrame({
     'make_model': model_names,
     'death_rate': death_rate,
     'death_rate_ci': death_rate_ci,
     'mv': mv,
     'sv': sv,
     'sv_roll': sv_roll,
     'model_years': model_years,
     'registered_years': registered_years
    })

In [5]:
# Additionally enter type of passenger vehicle for each row

for i in range(0,48):
    iihs_raw_death.loc[i, 'type'] = '4-Door Car'
for i in range(48,54):
    iihs_raw_death.loc[i, 'type'] = '2-Door Car'
for i in range(54,58):
    iihs_raw_death.loc[i, 'type'] = 'Sports Car'
for i in range(58,69):
    iihs_raw_death.loc[i, 'type'] = 'Luxury Car'
for i in range(69,81):
    iihs_raw_death.loc[i, 'type'] = 'Station Wagon'
for i in range(81,87):
    iihs_raw_death.loc[i, 'type'] = 'Minivan'
for i in range(87,162):
    iihs_raw_death.loc[i, 'type'] = 'SUV'
for i in range(162, 179):
    iihs_raw_death.loc[i, 'type'] = 'Luxury SUV'
for i in range(179,222):
    iihs_raw_death.loc[i, 'type'] = 'Pickup'

In [6]:
iihs_raw_death

,make_model,death_rate,death_rate_ci,mv,sv,sv_roll,model_years,registered_years,type
0,Mitsubishi Mirage hatchback,72,28-115,52,18,5,2014-17,"171,842",4-Door Car
1,Kia Rio,87,40-134,51,38,0,2014-17,"204,326",4-Door Car
2,Hyundai Accent,116,74-158,85,28,9,2014-17,"417,171",4-Door Car
3,Ford Fiesta,141,94-189,98,46,13,2014-17,"357,492",4-Door Car
4,Volkswagen Golf,0,0-34,0,0,0,2015-17,"108,084",4-Door Car
...,...,...,...,...,...,...,...,...,...
217,Ford F250 Crew Cab 4WD,30,6-55,15,15,0,2017,"150,292",Pickup
218,Ram 2500 Crew Cab short bed 4WD,30,18-43,13,17,6,2014-17,"580,038",Pickup
219,Ram 3500 Crew Cab long bed 4WD,32,12-52,19,13,3,2014-17,"239,085",Pickup
220,Ram 2500 Mega Cab 4WD,47,0-105,18,31,0,2014-17,"100,187",Pickup


In [7]:
# looking at models with duplicate make_model names
iihs_raw_death.groupby(by='make_model').count().sort_values(by='death_rate', ascending=False).head(10)

,death_rate,death_rate_ci,mv,sv,sv_roll,model_years,registered_years,type
make_model,,,,,,,,
Honda Accord,2,2,2,2,2,2,2,2
Ford Focus,2,2,2,2,2,2,2,2
Hyundai Accent,2,2,2,2,2,2,2,2
Ford Fiesta,2,2,2,2,2,2,2,2
Chevrolet Sonic,2,2,2,2,2,2,2,2
Mazda 3 hatchback,1,1,1,1,1,1,1,1
Kia Soul,1,1,1,1,1,1,1,1
Land Rover Range Rover 4WD,1,1,1,1,1,1,1,1
Land Rover Range Rover Evoque 4WD,1,1,1,1,1,1,1,1


In [8]:
# In all 4 out of 5 cases, there are sedan vs hatchback versions. For some reason IIHS defines the hatches as 'station wagons'
# For Honda Accord, it's 4-door sedan vs 2-door sedan

In [9]:
iihs_raw_death.loc[52, 'make_model'] = 'Honda Accord 2-door'
iihs_raw_death.loc[76, 'make_model'] = 'Ford Focus hatchback'
iihs_raw_death.loc[71, 'make_model'] = 'Hyundai Accent hatchback'
iihs_raw_death.loc[70, 'make_model'] = 'Ford Fiesta hatchback'
iihs_raw_death.loc[78, 'make_model'] = 'Chevrolet Sonic hatchback'

In [10]:
file = open('iihsdeath2017', 'wb')
pickle.dump(iihs_raw_death, file)
file.close()